# Ball Chain with Non Linearity

In the 1950's Enrico Fermi, Stan Ulam, and John Pasta used the MANIAC-1 computer to model a ball spring model with a weak non-linear force component.  They intended to use the model as a method to answer various questions related to irreversible statistical mechanics, but initial studies tested the most widely believed theories of equilibrium statistical mechanics like the equipartition of energy. In this notebook I will repeat their experiment with a quadratic force componen. Information about the model comes from Joseph Ford's paper [The Fermi-Pasta-Ulam Problem: Paradox Turns Discovery](http://www.cbpf.br/~cirto/MecEstNaoExten_HTML/AULAS/Aula_19/Ford_(The_Fermi_Pasta_Ulam_Problem_Paradox_Turns_Discovery)_%5BPhysics_Reports_1992%5D.pdf) 

If you are interested in reading [Fermi, Pasta, and Ulam's original paper](https://www.osti.gov/servlets/purl/4376203) click on the hyperlink.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ode
from vpython import * #yes, I know I'm not supposed to do this
from scipy.linalg import eigh
from scipy.linalg import eigh

ModuleNotFoundError: No module named 'numpy'

In the cell below I am creating a function that takes in the array 'y0' with the position and velocity of each ball.  It then applies the forces on each ball.  The first two equations are for the balls on the end of the string, and the last on is for the balls in between.

In [ ]:
def model_Natoms(y0, t): # y=[u,udot]; function returns derivative
    
    global N, k, alpha, m
    u = y0[0] #array of distances
    udot = y0[1] #array of velocities
        
    uddot = np.zeros(N)
    
    for i in range(0,N):
        if i==0:
            uddot[i] = -2*k/m*u[i] + k/m*u[i+1] + alpha*((0.0-u[i])**2-(u[i+1]-u[i])**2)
        elif i==N-1:
            uddot[i] = k/m*u[i-1] - 2*k/m*u[i] + alpha*((u[i-1]-u[i])**2-(0.0-u[i])**2)
        else:
            uddot[i] = k/m*u[i-1] -2*k/m*u[i] + k/m*u[i+1] + alpha*((u[i-1]-u[i])**2-(u[i+1]-u[i])**2)
            
    
    return np.array([udot, uddot])

This cell contains a function that iterates through each timestep using the RK45 integrator. To begin in the first normal mode and return to it this cell runs through 400,000 time steps.  It returns both the positions and velocity of each ball at each time step.

In [ ]:
def run_Natoms(Natoms, ks, mass, ui, udoti): 
    global N, k, m, h, alpha
    
    N = Natoms
    k = ks
    alpha = 0.25
    m = mass
    
#    u = ui #array
#    udot = udoti #array
    
    #t_total = 1000*np.pi
    t = 0
    #L=14
    Npoints=400000
    h = 0.03
    t_total = h*Npoints
    
    #lists
    ta = []
    ua = []
    udota = []

    ta.append(t)
    ua.append(ui)
    udota.append(udoti)

    #vector
    y0 = np.array([ui, udoti])
    
    for i in range(0,Npoints):
        
        #integrate
        y1 = ode.RK45(model_Natoms, y0, t, h) #update y[n]
                
        for n in range(N):
            y0[0][n] = y1[0][n]
        for n in range(N):
            y0[1][n] = y1[1][n]

        t = t + h #update clock
        
        ta.append(t)
        ua.append(y1[0])
        udota.append(y1[1])
                    
    return ta, ua, udota

This cell is running the function from above.

In [ ]:
Natoms=32
ks = 1.0
mass = 1.0
L = 1
L0 = L/(Natoms+1)

ui = np.zeros(Natoms)
udoti = np.zeros(Natoms)

for i in range(len(ui)):
    ui[i] = 1*np.sin(np.pi*(i+1)*L0)

ta, ulist, udotlist = run_Natoms(Natoms,ks,mass,ui,udoti)

ua=np.array(ulist)
udota = np.array(udotlist)

# plt.figure()
# plt.title("u of first atom")
# plt.plot(ta,ua[:,0], 'b-', label='u_1')
# plt.xlabel('t (arb)')
# plt.ylabel('u (arb)')
# plt.legend(loc='lower right')
# plt.show()

Here I am using Vpython to animate the ball-spring model.  By iterating through the position array returned from the 'run_Natoms' function I am able to observe the changes in the model.

In [5]:
def animate(x):
    scene=canvas()
    scene.background=color.white
    
    balls=[]
    xballs = np.linspace(-L/2+L0,L/2-L0,Natoms)
    
    ball = sphere(pos=vec(-L/2,0,0), radius=L0/4, color=color.red)
    for xb in xballs:
        ball = sphere(pos=vec(xb,0,0), radius=L0/4, color=color.red)
        balls.append(ball)
    ball = sphere(pos=vec(L/2,0,0), radius=L0/4, color=color.red)
        
    for i in range(len(x[:,0])):
        rate(2000)
        j=0
        while j<Natoms:
            balls[j].pos.y = x[i,j] 
            j = j + 1

In [6]:
animate(ua)

<IPython.core.display.Javascript object>